In [38]:
import pandas as pd 
import numpy as np 
import Tree
import math

In [312]:
data = pd.DataFrame([
        # 1
        ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.697, 0.460, '好瓜'],
        # 2
        ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.774, 0.376, '好瓜'],
        # 3
        ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.634, 0.264, '好瓜'],
        # 4
        ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.608, 0.318, '好瓜'],
        # 5
        ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.556, 0.215, '好瓜'],
        # 6
        ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.403, 0.237, '好瓜'],
        # 7
        ['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', 0.481, 0.149, '好瓜'],
        # 8
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', 0.437, 0.211, '好瓜'],
        # 9
        ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', 0.666, 0.091, '坏瓜'],
        # 10
        ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', 0.243, 0.267, '坏瓜'],
        # 11
        ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', 0.245, 0.057, '坏瓜'],
        # 12
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', 0.343, 0.099, '坏瓜'],
        # 13
        ['青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', 0.639, 0.161, '坏瓜'],
        # 14
        ['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', 0.657, 0.198, '坏瓜'],
        # 15
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.360, 0.370, '坏瓜'],
        # 16
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', 0.593, 0.042, '坏瓜'],
        # 17
        ['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', 0.719, 0.103, '坏瓜'],
    
    ],  columns = ['色泽', '根蒂', '敲击', '纹理', '脐部', '触感', '密度', '含糖率','好瓜'])

 

In [313]:
class DecisionTree:
    
    class normalTree(Tree.Tree):
        
        class _Node:
            __slots__ = '_element', '_parent', '_child', '_ty'
            def __init__(self, element, child = [], parent = None, ty = None):
                self._element = element 
                self._child = child
                self._parent = parent
                self._ty = ty  #当前节点内的
        
        def __init__(self):
            self.root = None
            self.size = 0
            
        def is_root(self):
            if self.root == None:
                return False
            else:
                return True
        
        def add_root(self, data):
            if self.is_root():
                raise ValueError('Root already exists')
            else:
                self.root = self._Node(data, child = [], parent = None, ty = None)
            
        def add_child(self, node, child):
            child._parent = node 
            node._child.append(child)
            self._size += 1
        
        def show_children_type(self, node):
            return [i._ty for i in node._child]

        
    
    def __init__(self, data):
        self.data = data
        self.size = len(data)
        self.feature = data.columns[0:-3]
        self.result = data.columns[-1]
        self.Tree = self.normalTree()
        
        
    def maxKind(self, data, feature): #计算一个特征里面最多的类
        num = 0
        j = -1
        kinds = list(set(data[self.result]))
        for i in range(len(kinds)):
            if len(data[data[self.result] == i]) > num:
                num = len(data[data[self.result] == kinds[i]])
                j = i
        
        return kinds[j]
        
        
    def Ent(self, data): #测算信息熵
        kinds = list(set(self.data[self.result]))
        ent = 0
        for i in kinds:
            s = len(data[data[self.result] == i])
            if len(data) == 0:
                ent = 0
            else:
                pk = s / len(data)
                if pk == 0:
                    ent += 0
                else:
                    ent += (-pk) * math.log2(pk)
            
        return ent
    
    def Gain(self, data, a):  #测算第a个特征带来的信息增益
        if a >= len(self.feature) or a < 0:
            raise ValueError('There is no such feature')
        
        feature = self.feature[a]
        
        featureKinds = list(set(self.data[feature]))
        
        gain = self.Ent(data)
        
        for i in featureKinds:
            subData = data[data[feature] == i]
            ent1 = self.Ent(subData)
            weight = len(subData) / len(data)
            gain -= weight * ent1
            
        return gain
            
    def BestFeature(self, data, features):   #使用信息增益选出下一个划分标准
        maxGain = 0
        idx = 0
        for i in range(len(features)):
            #print(self.Gain(data, i), maxGain)
            if self.Gain(data, i) > maxGain:
                maxGain = self.Gain(data, i)
                idx = i
            
        return idx
    
    def initTree(self):
        self.Tree.add_root(self.data)
        return self.Tree.root
    
    def GenerateTree(self, node, features):
        data = node._element
        
        cnt = 0
        for i in range(len(data[self.result]) - 1):
            if data[self.result][data.index[i]] == data[self.result][data.index[i+1]]:
                cnt += 0
            else:
                cnt += 1
        
                
        if cnt == 0:   
            node._ty = data[self.result][data.index[0]]
            return None
        
        if features == None:
            return None
            
        count = 0
        for i in features:
            if self.Gain(data, i) == 0:
                count += 0
            else:
                count += 1
       
            
        if count == 0:
            node._ty = self.maxKind(data, self.result)
            return None

        bestF = self.BestFeature(data, features) #找到最佳特征的索引
        node._ty = self.feature[bestF]
        
        featureKinds = list(set(self.data[self.feature[bestF]])) #单个特征里面的种类名称
        
        for kind in featureKinds:
            subdata = data[data[self.feature[bestF]] == kind]
            newNode = self.normalTree._Node(element = subdata, child = [], parent = node)
            node._child.append(newNode)
            
            if subdata.empty:
                newNode._ty = self.maxKind(data, self.result)
                
            else:
                self.GenerateTree(newNode, features)
            
        
        return None
        
        
            
            
    
    def excu(self):
        root = self.initTree()
        feature = [i for i in range(len(self.feature))]
        self.GenerateTree(root, feature)
        
        

            
        
        
        
        

        
        
        

In [314]:
dt = DecisionTree(data)
dt.excu()

In [315]:
dt.Tree.root._ty

'纹理'

In [316]:
dt.Tree.show_children_type(dt.Tree.root._child[2]._child[2])

['坏瓜', '触感', '好瓜']